In [1]:
def policy_gradient(S, R, gamma, p, V, policy, theta):

    while True:
        
        # policy evaluation
        while True:
            delta = 0
            for s in S:
                v = V[s]
                V[s] = sum([p[s, a, r, s2] * (r + gamma * V[s2]) for s2 in S for r in R])
                delta = max(delta, abs(v - V[s]))
            if delta < theta:
                break

        # policy improvement
        policy_stable = True
        for s in S:
            old_action = policy[s]
            policy[s] = max(S, key = lambda a: sum([p[s, a, r, s2] * (r + gamma * V[s2]) for s2 in S for r in R]))
            if old_action != policy[s]:
                policy_stable = False
        if policy_stable:
            break
    
    return V, policy
